In [88]:
from pydantic import TypeAdapter, BaseModel, Field
from typing import Literal
import json
from typing import Annotated, Union


class Cat(BaseModel):
    type: Literal["cat"] = "cat"
    hairless: bool = False


class Dog(BaseModel):
    type: Literal["dog"] = "dog"
    breed: str


FooBar = TypeAdapter(Annotated[Union[Cat, Dog], Field(discriminator="type")])

child = FooBar.validate_json(json.dumps({"type": "cat", "hairless": True}))

print(child)
print(type(child))

type='cat' hairless=True
<class '__main__.Cat'>


In [103]:
foobar = Annotated[Union["BranchA", "BranchB", "Leaf"], Field(discriminator="type")]


class Leaf(BaseModel):
    type: Literal["fooo"] = "fooo"
    leaf: str


class BranchA(BaseModel):
    type: Literal["branchA"] = "branchA"
    branches: list[foobar]


class BranchB(BaseModel):
    type: Literal["branchB"] = "branchB"
    branches: list[foobar]


FooBar = TypeAdapter(foobar)

example_1_raw = {
    "type": "branchA",
    "branches": [
        {"type": "fooo", "leaf": "leaf1"},
        {"type": "branchB", "branches": [{"type": "fooo", "leaf": "leaf2"}]},
    ],
}
# example_1_raw = {
#     "type": "branchA",
#     "branches": [
#         {
#             "type": "branchB",
#             "branches": [
#                 {
#                     "type": "fooo",
#                     "leaf": "leaf2"
#                 },
#                 {
#                     "type": "fooo",
#                     "leaf": "leaf2"
#                 }
#             ]
#         },
#                 {
#                     "type": "fooo",
#                     "leaf": "leaf2"
#                 }]}

example_1 = FooBar.validate_json(json.dumps(example_1_raw))
print(example_1)

type='branchA' branches=[Leaf(type='fooo', leaf='leaf1'), BranchB(type='branchB', branches=[Leaf(type='fooo', leaf='leaf2')])]


In [1]:
from enum import Enum
from pydantic import BaseModel
from pydantic import TypeAdapter, BaseModel, Field
from typing import Annotated, Union


# enums
class MessageRole(str, Enum):
    USER = "user"
    SYSTEM = "system"
    ASSISTANT = "assistant"


class TemplateItemType(str, Enum):
    MESSAGE = "message"
    TEXT = "text"
    EVAL = "eval"
    LOOP = "loop"


# openai message
class Message(BaseModel):
    role: MessageRole
    content: str


# template items
TemplateItemUnion = Annotated[
    Union["TemplateMessage", "TemplateText", "TemplateEval", "TemplateLoop"],
    Field(discriminator="type"),
]
TemplateItem = TypeAdapter(TemplateItemUnion)


# message
class TemplateMessage(BaseModel):
    type: TemplateItemType = TemplateItemType.MESSAGE
    role: MessageRole
    template: list[TemplateItemUnion]

    # @classmethod
    # def model_validate(cls, data):
    #     template = []
    #     for item in data["template"]:
    #         template.append(TemplateItem.model_validate(item))
    #     return cls(**{**data, "template": template})


# text
class TemplateText(BaseModel):
    type: TemplateItemType = TemplateItemType.TEXT
    content: str


# eval
class TemplateEval(BaseModel):
    type: TemplateItemType = TemplateItemType.EVAL
    value: str


# loop
class TemplateLoop(BaseModel):
    type: TemplateItemType = TemplateItemType.LOOP
    iterator: str
    iterable: str
    template: list[TemplateItemUnion]

    # @classmethod
    # def model_validate(cls, data):
    #     template = []
    #     for item in data["template"]:
    #         template.append(TemplateItem.model_validate(item))
    #     return cls(**{**data, "template": template})


# template
class Template(BaseModel):
    items: list[TemplateItemUnion]

PydanticUndefinedAnnotation: name 'TemplateMessage' is not defined

For further information visit https://errors.pydantic.dev/2.6/u/undefined-annotation